In [1]:
import requests
from datetime import *
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd

import gurobi as gb
from gurobipy import GRB

import networkx as nx

import math
from statistics import mean 
from itertools import product

import subprocess
import pandas as pd

import sys 
import os
path = os.path.dirname(os.getcwd())
sys.path.append(path)
from lib import *

In [2]:
# Cloud Application Architecture data
data, vars_ms, userMax, ed, q = getConstantsFromBPMN('../flightBooking.json')

# Read user data for each hour (Wikipedia clickData from 2015)
df = pd.read_csv(r'../data/pprojectcount_wikiDE_2015.csv')
clickData2015_hourly = df["De"]
users = clickData2015_hourly

# Read carbon intensity data for each hour
df = pd.read_csv(r'../data/DE_2020.csv') # in gCO2eq per kWh
ci_2020 = df['carbon_intensity_avg']
df = pd.read_csv(r'../data/DE_2021.csv') # in gCO2eq per kWh
ci_2021 = df['carbon_intensity_avg']

cb = calcCarbonBudgetFrom_AVG_CE()

numHours = 8760

In [3]:
# Premodifier (Aim: Efficiency)
vars_gb = [[]]


indices = []
for ms in data['components']:
    indices.append([0] * len(data['components'][ms]))

for i in indices:
    vars_gb *= len(i)

for i in range(len(vars_gb)):
    ed_combo = list(product(*ed))
    userMax_combo = list(product(*userMax))
    q_combo = list(product(*q))
    vars_gb[i].append([ed_combo[i], userMax_combo[i], q_combo[i]])


sf = [[] for h in range(numHours)]
for h in range(numHours):
    sf[h] = [[] for i in range(len(vars_gb))]
    for i in range(len(vars_gb)):
        sf[h][i] = [[] for ms in range(len(indices))]
        for ms in range(len(indices)):
            if ms == 0:
                sf[h][i][ms] = math.ceil(users[h] / vars_gb[0][i][1][ms])
            elif vars_gb[0][i][0][ms] == 0:
                sf[h][i][ms] = 0
            else:
                sf[h][i][ms] = math.ceil((sf[h][i][ms-1] * math.prod(vars_gb[0][i][2][0:ms])) / vars_gb[0][i][1][ms])
    



In [4]:
# Optimizer configuration, when all data for the whole year is known beforehand
opt_mod = gb.Model(name="globalView")
# Create optimization model
m = gb.Model('ca_microservice_global')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-23


In [5]:
b = m.addVars([(h,i) for h in range(numHours) for i in range(len(vars_gb))],vtype=GRB.BINARY,name="b")

In [6]:
for h in range(numHours):
    m.addConstr(gb.quicksum(b[h,i] for i in range(len(vars_gb))) == 1, name="c-ms"+str(ms)+str(h))


m.addConstr(gb.quicksum(b[h,i]*sum(sf[h][i][ms]*vars_gb[0][i][0][ms] for ms in range(len(indices)))*ci_2021[h] for h in range(numHours) for i in range(len(vars_gb))) <= cb)

m.update()

In [7]:
obj = gb.quicksum(b[h,i]*math.prod(vars_gb[0][i][2][0:len(indices)])*users[h] for h in range(numHours) for i in range(len(vars_gb))) 
m.setObjective(obj, GRB.MAXIMIZE)

m.tune()
m.optimize()


Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 8761 rows, 315360 columns and 630648 nonzeros
Model fingerprint: 0x13417b25
Variable types: 0 continuous, 315360 integer (315360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+07]
  Objective range  [7e+03, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 3.780541e+09
Presolve removed 2 rows and 262811 columns
Presolve time: 0.55s
Presolved: 8759 rows, 52549 columns, 105098 nonzeros
Variable types: 0 continuous, 52549 integer (52549 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 6.791763e+09, 883 i

# Save to csv

In [8]:
f = open('/home/k/projects/CA_microservices/results/baselineOptimization_result.csv', 'a')
writer = csv.writer(f)
for (i, j),v in b.items():
    if v.X == 1.0:
        row = []
        energyDemand = sum([vars_gb[0][j][0][ms]*sf[i][j][ms] for ms in range(len(indices))])
        u = math.prod(vars_gb[0][j][2][0:len(indices)])*users[i]
        carbonEmission = energyDemand * ci_2021[i]
        row.append(u)
        row.append(energyDemand)
        row.append(carbonEmission)
        writer.writerow(row)
f.close()